In [2]:
from nbmultitask import ThreadWithLogAndControls

In [33]:
from threading import Thread
import audio
import video
import nlp
import time
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
A.playWav()
print("hello")

FORMAT: 8 
CHANNELS: 1 
RATE: 16000
hello


In [32]:
nlp.text2sound("孙猴子！")
A.playWav()

FORMAT: 8 
CHANNELS: 1 
RATE: 16000


In [25]:
A = audio.Audio(600)
A.streamStart()
A.playWav("audio/record.wav")

FORMAT: 8 
CHANNELS: 1 
RATE: 16000


In [3]:
g_audio_record_start = False
g_audio_available = False
g_emotion = None
g_emotion_captured = False
g_listen_lock = False

def capture():
    global g_emotion
    global g_emotion_captured 
    g_emotion_captured = False
    for i in range(5):
        print("trying to analyze emotions.. attempt " + str(i))
        frame = C.shoot_test('o.jpg')
        e = P.emotion_dect(frame)
        if e != None:
            g_emotion = e
            g_emotion_captured = True
            return 
    print("fail to dect emotions")

def listen():
    global g_audio_record_start
    global g_audio_available
    while 1:
        if A.state == 'silent' and g_listen_lock == False:
            A.waiting()
            g_audio_record_start = True
        elif A.state == 'recording' and g_listen_lock == False:
            A.record()
            g_audio_record_start = False
            g_audio_available = True

In [ ]:
C = video.Camera()
P = video.PA()        
A = audio.Audio(600)
A.streamStart()

T_audio = Thread(target = listen)
T_audio.start()

state = 'waiting'
while 1:
    if state == 'waiting':
        if g_audio_record_start == True:
            T_video = Thread(target = capture)
            T_video.start()
            state = 'recording'
            print("state chanegd to recording")
    
    if state == 'recording':
        if g_audio_available == True:
            g_audio_available = False
            print("start stt...")
            g_listen_lock = True
#             text = nlp.sound2text(A.file_path)
            time.sleep(0.4)
            print("stt done. state changed to processing")
            state = 'processing'
    
    if state == 'processing':
        if g_emotion_captured == True:
            print(g_emotion_captured)
#         text = text
        text = '你好'
        re_text = nlp.tuling(text)
        state = 'reply'
    
    if state == 'reply':
#         nlp.text2sound(re_text)
#         speek
        time.sleep(0.4)
        state = 'waiting'
        g_listen_lock = False
        print("back to waiting")

start recording...
trying to analyze emotions.. attempt 0
state chanegd to recording
silent...10
silent...10
silent...20
silent...30
record session closed.
wave saved...
start stt...
stt done. state changed to processing
你好
emotion analyze time cost 1.8489775657653809
你好，以后请多指教
back to waitingstart recording...

trying to analyze emotions.. attempt 0
state chanegd to recording
emotion analyze time cost 0.055780649185180664
silent...10
silent...10
silent...10
silent...20
silent...30
record session closed.
wave saved...
start stt...
start recording...
silent...10
stt done. state changed to processing
True
你好
好吧，你也好。
silent...10
silent...20
back to waiting
trying to analyze emotions.. attempt 0state chanegd to recording

emotion analyze time cost 0.0959162712097168
silent...10
silent...20
silent...10
silent...10
silent...10
silent...20
silent...30
record session closed.
wave saved...
start stt...
stt done. state changed to processing
True
你好
你也好～
back to waitingstart recording...

trying 